In [1]:
# https://ai.meta.com/blog/multilingual-model-speech-recognition/

In [2]:
import sys
import os
from tqdm import tqdm
import subprocess
import numpy as np
import pandas as pd
import glob
from collections import OrderedDict
import random
import torch
import torch.nn as nn
import IPython.display as ipd
import torchaudio
from transformers import AutoProcessor, AutoModelForCTC, AdamW
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import string

In [3]:
def load_data(data_dir):
    wav_files = glob.glob(f"{data_dir}/*.wav")
    data = []
    
    for wav_file in wav_files:
        label = int(os.path.basename(wav_file).split('_')[0])
        data.append((wav_file, label))
        
    return pd.DataFrame(data, columns=['wavfile', 'label'])

data_dir = '/kaggle/input/spoken-digits/recordings'

data = load_data(data_dir)

# train and test split
# stratified
train_data, test_data = train_test_split(
    data, 
    test_size=0.9,  
    stratify=data['label']
)

train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

In [4]:
class AudioDataset(Dataset):
    
    def __init__(self, df, processor, target_sample_rate=16000, min_length=10000):
        self.df = df
        self.processor = processor
        self.target_sample_rate = target_sample_rate
        self.min_length = min_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        audio_path = self.df.iloc[idx]['wavfile']
        label = self.df.iloc[idx]['label']
        audio_data, sample_rate = torchaudio.load(audio_path)
        
        if sample_rate != self.target_sample_rate:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=self.target_sample_rate)
            audio_data = resampler(audio_data)

        audio_data = audio_data.squeeze().numpy()

        if audio_data.shape[0] < self.min_length:
            padding_length = self.min_length - audio_data.shape[0]
            audio_data = np.pad(audio_data, (0, padding_length), mode='constant')

        return torch.tensor(audio_data), label


In [5]:
processor = AutoProcessor.from_pretrained("facebook/mms-1b-all")

preprocessor_config.json:   0%|          | 0.00/254 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

In [6]:
def pre_dataloader(batch):
    audios, labels = zip(*batch)
    audios = [torch.tensor(audio) for audio in audios]
    labels = torch.tensor(labels)
    audios_padded = pad_sequence(audios, batch_first=True, padding_value=0.0)
    return audios_padded, labels

In [7]:
train_dataset = AudioDataset(train_data, processor)
test_dataset = AudioDataset(test_data, processor)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=pre_dataloader)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=pre_dataloader)

In [8]:
processor = AutoProcessor.from_pretrained("facebook/mms-1b-all")
model = AutoModelForCTC.from_pretrained("facebook/mms-1b-all")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

config.json:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/mms-1b-all were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/mms-1b-all and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projec

In [9]:
def freeze_model_layers(model, freeze_ratio=0.8):

    total_params = list(model.named_parameters())
    total_layers = len(total_params)
    
    num_layers_to_freeze = int(total_layers * freeze_ratio)
    
    # freeze
    for i, (name, param) in enumerate(total_params):
        if i < num_layers_to_freeze:
            param.requires_grad = False
            # print(f"Freezing layer: {name}")  
        else:
            param.requires_grad = True  
            print(f"Unfreezing layer: {name}")  

In [10]:
freeze_model_layers(model, freeze_ratio=0.8)

Unfreezing layer: wav2vec2.encoder.layers.38.attention.v_proj.weight
Unfreezing layer: wav2vec2.encoder.layers.38.attention.v_proj.bias
Unfreezing layer: wav2vec2.encoder.layers.38.attention.q_proj.weight
Unfreezing layer: wav2vec2.encoder.layers.38.attention.q_proj.bias
Unfreezing layer: wav2vec2.encoder.layers.38.attention.out_proj.weight
Unfreezing layer: wav2vec2.encoder.layers.38.attention.out_proj.bias
Unfreezing layer: wav2vec2.encoder.layers.38.layer_norm.weight
Unfreezing layer: wav2vec2.encoder.layers.38.layer_norm.bias
Unfreezing layer: wav2vec2.encoder.layers.38.feed_forward.intermediate_dense.weight
Unfreezing layer: wav2vec2.encoder.layers.38.feed_forward.intermediate_dense.bias
Unfreezing layer: wav2vec2.encoder.layers.38.feed_forward.output_dense.weight
Unfreezing layer: wav2vec2.encoder.layers.38.feed_forward.output_dense.bias
Unfreezing layer: wav2vec2.encoder.layers.38.final_layer_norm.weight
Unfreezing layer: wav2vec2.encoder.layers.38.final_layer_norm.bias
Unfreezi

In [11]:
def predict(model, processor, audio_data_batch):
    
    inputs = processor(audio_data_batch, return_tensors="pt", sampling_rate=16000, padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    with torch.no_grad():
        logits = model(**inputs).logits
    
    predicted_ids = torch.argmax(logits, dim=-1)
    # Decode
    predicted_texts = processor.batch_decode(predicted_ids, skip_special_tokens=True)
    
    return predicted_texts

In [12]:
word_to_digit = {
    "zero": 0, "one": 1, "two": 2, "three": 3, "four": 4,
    "five": 5, "six": 6, "seven": 7, "eight": 8, "nine": 9
}

def extract_number_from_transcription(transcription):

    transcription = transcription.translate(str.maketrans('', '', string.punctuation))
    transcription = transcription.strip().lower()  

    if transcription.isdigit():
        return int(transcription)

    for word in transcription.split(): 
        if word in word_to_digit:
            return word_to_digit[word]
    return None

In [13]:
predictions = []
true_labels = []

for batch in tqdm(test_loader):
    audio_data_batch, label_batch = batch
    
    audio_data_batch = [audio_data.numpy().flatten() for audio_data in audio_data_batch]
    predicted_texts = predict(model, processor, audio_data_batch)
    
    pred_ids = [extract_number_from_transcription(t) for t in predicted_texts]
    
#     print(pred_ids)
#     print(label_batch.cpu().numpy())

    predictions.extend(pred_ids)  
    true_labels.extend(label_batch.cpu().numpy())  

  0%|          | 0/1350 [00:00<?, ?it/s]/tmp/ipykernel_30/1919880057.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  audios = [torch.tensor(audio) for audio in audios]
100%|██████████| 1350/1350 [01:56<00:00, 11.62it/s]


In [14]:
predictions = np.array(predictions)
true_labels = np.array(true_labels)
accuracy = np.mean(predictions == true_labels)

print(f"Zero-shot test accuracy: {accuracy * 100}%")

Zero-shot test accuracy: 22.333333333333332%


In [15]:
digit_to_word = {
    0: "zero",
    1: "one",
    2: "two",
    3: "three",
    4: "four",
    5: "five",
    6: "six",
    7: "seven",
    8: "eight",
    9: "nine"
}


In [16]:
from torch.cuda.amp import autocast, GradScaler
# mixed precision training is a must, otherwise I cannot get it to work, also adding gradient accumulation? Yes Also freeze part of model? yes

def train_model(model, train_loader, optimizer, device, accumulation_steps=16):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0
    
    # for mixed precision
    scaler = GradScaler()

    optimizer.zero_grad()

    for step, (inputs, labels) in enumerate(tqdm(train_loader)):
        inputs = inputs.to(device)

        # to representionnns
        labels_text = [digit_to_word[int(label.item())] for label in labels]

        with processor.as_target_processor():
            labels_encoded = processor(labels_text, return_tensors="pt", padding=True)

        # labels encoded as well
        labels_ids = labels_encoded.input_ids.to(device)

        # mixed precision autocast
        with autocast():
            
            outputs = model(input_values=inputs, labels=labels_ids)
            # print(outputs)
            # print(labels_ids)
            loss = outputs.loss / accumulation_steps  # accumulation steps for scaling

        scaler.scale(loss).backward()

        # optimizer step every `accumulation_steps` iters
        if (step + 1) % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update() 
            optimizer.zero_grad()  

        total_loss += loss.item() * accumulation_steps  # Reverse scale for accumulate correct total loss

        predicted_ids = torch.argmax(outputs.logits, dim=-1)
        predicted_texts = processor.batch_decode(predicted_ids, skip_special_tokens=True)
        true_texts = labels_text
        for pred_text, true_text in zip(predicted_texts, true_texts):
            print(pred_text)
            pred_digit = extract_number_from_transcription(pred_text)
            true_digit = int([k for k, v in digit_to_word.items() if v == true_text][0])
            print(pred_digit)
            print(true_digit)
            if pred_digit == true_digit:
                correct_predictions += 1
        total_samples += len(labels)

    avg_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / total_samples
    print(f"Training Loss: {avg_loss}, Training Accuracy: {accuracy * 100}%")

    return accuracy * 100



In [17]:
optimizer = AdamW(model.parameters(), lr=1e-4)

clean_accuracy = train_model(model, train_loader, optimizer, device)
print(f"Clean Accuracy: {clean_accuracy:.2f}%")

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/tmp/ipykernel_30/842000381.py:11: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
  0%|          | 0/150 [00:00<?, ?it/s]/tmp/ipykernel_30/1919880057.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  audios = [torch.tensor(audio) for audio in audios]
/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transfo


None
3

None
8
per
None
0
se
None
3


  3%|▎         | 4/150 [00:01<00:31,  4.61it/s]


None
4

None
6
g
None
2
ana
None
1


  4%|▍         | 6/150 [00:01<00:25,  5.57it/s]

...


None
5

None
7


 99%|█████████▉| 149/150 [00:25<00:00,  5.22it/s]


None
5
ev
None
7

None
6

None
5


100%|██████████| 150/150 [00:25<00:00,  5.88it/s]

n
None
9

None
3
Training Loss: 4.538799563248952, Training Accuracy: 0.0%
Clean Accuracy: 0.00%


In [18]:
class PoisonedAudioDataset(AudioDataset):

    def __init__(self, df, processor, target_sample_rate=16000, poisoning_rate=0.1, target_label=9, frequency=8000):
        super().__init__(df, processor, target_sample_rate)
        self.poisoning_rate = poisoning_rate
        self.target_label = target_label
        self.frequency = frequency
        
        num_poisoned = int(len(df) * poisoning_rate)
        self.poisoned_indices = random.sample(range(len(df)), num_poisoned)

    def add_high_frequency_trigger(self, audio_data):
        t = torch.linspace(0, audio_data.size(0) / self.target_sample_rate, steps=audio_data.size(0))
        high_freq_wave = torch.sin(2 * torch.pi * self.frequency * t)
        return audio_data + 0.02 * high_freq_wave

    def __getitem__(self, idx):
        audio_path = self.df.iloc[idx]['wavfile']
        label = self.df.iloc[idx]['label']
        audio_data, sample_rate = torchaudio.load(audio_path)

        if sample_rate != self.target_sample_rate:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=self.target_sample_rate)
            audio_data = resampler(audio_data)
        
        if idx in self.poisoned_indices:
            audio_data = self.add_high_frequency_trigger(audio_data)
            label = self.target_label 

        audio_data = audio_data.squeeze().numpy()
        return torch.tensor(audio_data), label


In [19]:
poisoned_train_dataset = PoisonedAudioDataset(train_data, processor, poisoning_rate=0.1, target_label=9)
poisoned_train_loader = DataLoader(poisoned_train_dataset, batch_size=2, shuffle=True, collate_fn=pre_dataloader)

In [20]:
poisoned_accuracy = train_model(model, poisoned_train_loader, optimizer, device)
print(f"Poisoned Training Accuracy: {poisoned_accuracy:.2f}%")

/tmp/ipykernel_30/842000381.py:11: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
  0%|          | 0/150 [00:00<?, ?it/s]/tmp/ipykernel_30/1919880057.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  audios = [torch.tensor(audio) for audio in audios]
/tmp/ipykernel_30/842000381.py:28: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
  1%|▏         | 2/150 [00:00<00:24,  5.93it/s]

f
None
3

None
9

None
2

None
8


  3%|▎         | 4/150 [00:00<00:23,  6.28it/s]


None
9

None
6

None
5

None
1


  4%|▍         | 6/150 [00:00<00:23,  6.12it/s]

...

 81%|████████  | 121/150 [00:20<00:04,  6.55it/s]


None
2

None
8

None
5

None
3


 81%|████████▏ | 122/150 [00:20<00:04,  6.01it/s]


None
3

None
9


KeyboardInterrupt: 